<a href="https://colab.research.google.com/github/urness/CS167Code/blob/main/Day05_Notes_kNN_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Day 5 Notes: k Nearest Neighbors
The following code includes the steps necessary to build a k nearest neighbors model. We'll go through each step, and then I'll ask you to pull these steps together into a function that you can pass a new observation of a flower in to and it will output the species the model predicts that flower is.

In [4]:
import pandas
from google.colab import drive
drive.mount('/content/drive')
#go through authentication step

Mounted at /content/drive


In [5]:
#import the data and take a look at it
data = pandas.read_csv('/content/drive/MyDrive/CS167Spring22/datasets/irisData.csv')
data.head()

,sepal length,sepal width,petal length,petal width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [ ]:
data['petal width'].mean() #get the average of a column

1.1986666666666672

In [ ]:
# Calculate the distance
import numpy
#adding a new column
data['distance_to_new'] = numpy.sqrt(
    (5.1-data['petal length'])**2
    +(7.2-data['sepal length'])**2
    +(1.5-data['petal width'])**2
    +(2.5-data['sepal width'])**2)
data

,sepal length,sepal width,petal length,petal width,species,distance_to_new
0,5.1,3.5,1.4,0.2,Iris-setosa,4.559605
1,4.9,3.0,1.4,0.2,Iris-setosa,4.573839
2,4.7,3.2,1.3,0.2,Iris-setosa,4.782259
3,4.6,3.1,1.5,0.2,Iris-setosa,4.665833
4,5.0,3.6,1.4,0.2,Iris-setosa,4.629255
...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica,1.072381
146,6.3,2.5,5.0,1.9,Iris-virginica,0.989949
147,6.5,3.0,5.2,2.0,Iris-virginica,1.000000
148,6.2,3.4,5.4,2.3,Iris-virginica,1.593738


In [ ]:
# Sort the data and display it
sorted_data = data.sort_values(['distance_to_new']) 
sorted_data #display it

,sepal length,sepal width,petal length,petal width,species,distance_to_new
76,6.8,2.8,4.8,1.4,Iris-versicolor,0.591608
52,6.9,3.1,4.9,1.5,Iris-versicolor,0.700000
77,6.7,3.0,5.0,1.7,Iris-versicolor,0.741620
50,7.0,3.2,4.7,1.4,Iris-versicolor,0.836660
129,7.2,3.0,5.8,1.6,Iris-virginica,0.866025
...,...,...,...,...,...,...
8,4.4,2.9,1.4,0.2,Iris-setosa,4.835287
38,4.4,3.0,1.3,0.2,Iris-setosa,4.921382
42,4.4,3.2,1.3,0.2,Iris-setosa,4.945705
22,4.6,3.6,1.0,0.2,Iris-setosa,5.144900


In [ ]:
# give the top 5 closest neighbors
sorted_data.iloc[0:5]

,sepal length,sepal width,petal length,petal width,species,distance_to_new
76,6.8,2.8,4.8,1.4,Iris-versicolor,0.591608
52,6.9,3.1,4.9,1.5,Iris-versicolor,0.700000
77,6.7,3.0,5.0,1.7,Iris-versicolor,0.741620
50,7.0,3.2,4.7,1.4,Iris-versicolor,0.836660
129,7.2,3.0,5.8,1.6,Iris-virginica,0.866025


In [ ]:
# give the most commonly occuring species of the 5 closest neighbors
sorted_data.iloc[0:5]['species'].mode()
#sorted_data.iloc[0:5]['species'].mode()[0] #return just the string

0    Iris-versicolor
dtype: object

## Exercise:
Now it's your turn. Use the code snippets from above to build out a kNN function that takes in `specimen`, `data`, and `k` and ouputs the prediction. 

In [8]:
import numpy
def knn(specimen, data, k):
  # your code here
  data_copy = data.copy()
  data_copy['distance_to_new'] = numpy.sqrt(
    (specimen['petal length']-data['petal length'])**2
    +(specimen['sepal length']-data['sepal length'])**2
    +(specimen['petal width']-data['petal width'])**2
    +(specimen['sepal width']-data['sepal width'])**2)
  sorted_data = data_copy.sort_values(['distance_to_new']) 
  top_k = sorted_data.iloc[0:k]
  print(top_k['species'].mode())


  

In [9]:
# Use the code below to build a dictionary of values and test your new function:
iris = {}
iris['petal length'] = 5.1
iris['sepal length'] = 7.2
iris['petal width'] = 1.5
iris['sepal width'] = 2.5
# 5-NN 
knn(iris,data,5)

0    Iris-versicolor
dtype: object


## Exercise #2:
Now it's your turn. Use the code snippets from above to build out a *weighted* kNN function that takes in `specimen`, `data`, and `k` and ouputs the prediction. 

In [17]:
import numpy

# weighted kNN
def w_knn(specimen, data, k):
  # calculate the distance
  data_copy = data.copy()
  data_copy['distance_to_new'] = numpy.sqrt(
    (specimen['petal length']-data['petal length'])**2
    +(specimen['sepal length']-data['sepal length'])**2
    +(specimen['petal width']-data['petal width'])**2
    +(specimen['sepal width']-data['sepal width'])**2)
  
  # calculate the weights (remember, the weights are 1/d^2)
  data_copy['weights'] = 1/(data_copy['distance_to_new']**2)

  # find the closest k
  sorted_data = data_copy.sort_values(['weights'], ascending = False) 
  top_k = sorted_data.iloc[0:k]

  # use groupby to sum the weights of each species in the closest k
  sum_of_weights_per_species = top_k.groupby(['species'])['weights'].sum()

  sorted = sum_of_weights_per_species.sort_values(ascending=False)
  return sorted.keys()[0]
  # return the class that has the larger sum of weights

  # idxmax() -- nah

In [18]:
iris = {}
iris['petal length'] = 5.1
iris['sepal length'] = 7.2
iris['petal width'] = 1.5
iris['sepal width'] = 2.5
# 5-NN 
print(w_knn(iris,data,5))


     sepal length  sepal width  ...  distance_to_new   weights
76            6.8          2.8  ...         0.591608  2.857143
52            6.9          3.1  ...         0.700000  2.040816
77            6.7          3.0  ...         0.741620  1.818182
50            7.0          3.2  ...         0.836660  1.428571
129           7.2          3.0  ...         0.866025  1.333333

[5 rows x 7 columns]
Iris-versicolor
